In [2]:
import pandas
from nltk import agreement

## Rating data

In [25]:
# Load rating data
df_rating = pandas.read_csv('../rawdata/accounts_rating_final.csv').iloc[:5082,:]
df_rating.head()

,id,rate1,rater1,rater1_diff,rate1_final,rate2,rater2,rater2_diff,rate2_final,final
0,1,2,r1,2,2,2,r3,1,2,2
1,2,1,r1,1,1,1,r3,1,1,1
2,3,2,r3,1,2,2,r1,2,2,2
3,4,3,r3,1,1,3,r1,0,1,1
4,5,3,r3,3,3,2,r4,2,3,3


## Interrater agreement round 1

In [26]:
rate1 = df_rating.rate1.values
rate2 = df_rating.rate2.values

taskdata=[[0,str(i),str(rate1[i])] for i in range(0,len(rate1))]+[[1,str(i),str(rate2[i])] for i in range(0,len(rate2))]
ratingtask = agreement.AnnotationTask(data=taskdata)
print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
print("alpha " +str(ratingtask.alpha()))
print("scotts " + str(ratingtask.pi()))

kappa 0.8469024655382728
fleiss 0.8469024655382728
alpha 0.8469096984313795
scotts 0.8468946349361941


In [35]:
print('first round')
print('Equally rated as bot: ',df_rating[lambda x:x['rate1'] == x['rate2']][lambda x: x['rate1']==2].shape[0])
print('Equallry rated human: ',df_rating[lambda x:x['rate1'] == x['rate2']][lambda x: x['rate1']==1].shape[0])
print('Equallry rated IDK: ',df_rating[lambda x:x['rate1'] == x['rate2']][lambda x: x['rate1']==3].shape[0])
print('Rated differently: ',df_rating[lambda x:x['rate1'] != x['rate2']].shape[0])
print('--------------------')
print('second round')
print('Equally rated as bot: ',df_rating[lambda x:x['rate1_final'] == x['rate2_final']][lambda x: x['rate1_final']==2].shape[0])
print('Equallry rated human: ',df_rating[lambda x:x['rate1_final'] == x['rate2_final']][lambda x: x['rate1_final']==1].shape[0])
print('Equallry rated IDK: ',df_rating[lambda x:x['rate1_final'] == x['rate2_final']][lambda x: x['rate1_final']==3].shape[0])
print('Rated differently: ',df_rating[lambda x:x['rate1_final'] != x['rate2_final']].shape[0])

first round
Equally rated as bot:  472
Equallry rated human:  4364
Equallry rated IDK:  69
Rated differently:  177
--------------------
second round
Equally rated as bot:  527
Equallry rated human:  4473
Equallry rated IDK:  77
Rated differently:  4


# Interrater agreement after discussion (round 2)

In [4]:
rate1 = df_rating.rate1_final.values
rate2 = df_rating.rate2_final.values

taskdata=[[0,str(i),str(rate1[i])] for i in range(0,len(rate1))]+[[1,str(i),str(rate2[i])] for i in range(0,len(rate2))]
ratingtask = agreement.AnnotationTask(data=taskdata)
print("kappa " +str(ratingtask.kappa()))
print("fleiss " + str(ratingtask.multi_kappa()))
print("alpha " +str(ratingtask.alpha()))
print("scotts " + str(ratingtask.pi()))

kappa 0.9963463019414441
fleiss 0.9963463019414441
alpha 0.9963466420794419
scotts 0.9963463016304697


# Computing time spent for rating

In [5]:
df_ratingtime = pandas.read_csv('../rawdata/ratedata.csv',parse_dates=['rated_at'])

In [6]:
df_ratingtime['prated_at'] = df_ratingtime['rated_at'].shift(-1)
df_ratingtime =(
    df_ratingtime
    .assign(
        time_interval = lambda x: pandas.np.where(
            ((x['prated_at']-x['rated_at']).dt.total_seconds()>60) | ((x['prated_at']-x['rated_at']).dt.total_seconds()<0),
            60, (x['prated_at']-x['rated_at']).dt.total_seconds())
    )
)

In [7]:
df_ratingtime.groupby('rater').agg({'time_interval':lambda x: x.sum()/3600})

,time_interval
rater,
12,28.416667
13,5.929444
14,16.169444
15,11.825278


### Time spent on rating + time spend on discussion sessions with raters

In [8]:
print('Total time spent on rating by raters: ',df_ratingtime.time_interval.sum()/3600)
print('Time spend on discussion sessions: ')
print('Discussion session 1: 3 hours, 4 raters, total of 12 hours')
print('Discussion session 2: 3 hours, 4 raters, total of 12 hours')
print('Discussion session 2: 2 hours, 4 raters, total of 8 hours')
print('Discussion session 3: 2 hours, 3 raters, total of 6 hours')
print('Discussion session 4: 1.5 hours, 3 raters, total of 4.5 hours')
print('\nTotal time spent: ',str(62.34 + 12+12+8+6+4.5))

Total time spent on rating by raters:  62.340833333333336
Time spend on discussion sessions: 
Discussion session 1: 3 hours, 4 raters, total of 12 hours
Discussion session 2: 3 hours, 4 raters, total of 12 hours
Discussion session 2: 2 hours, 4 raters, total of 8 hours
Discussion session 3: 2 hours, 3 raters, total of 6 hours
Discussion session 4: 1.5 hours, 3 raters, total of 4.5 hours

Total time spent:  104.84
